In [4]:
import cv2
import numpy as np
import os
import json
import ast

In [5]:
pixel_to_um = 0.1155
pixel_to_m = pixel_to_um * 1e-6

In [6]:
# Make a list of all folders in a folder
def get_folder_list(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [7]:
def get_BB(folder, set_list):
    bbox_dict = {}
    for s in set_list[0:3]:
        seq = cv2.VideoCapture(f"{folder}/{s}/Image#%04d.jpg")
        if not seq.isOpened():
            print('error reading image sequence')
            break
        valid, frame = seq.read()
        if not valid:
            print('error reading image')
            break
        bbox = cv2.selectROIs("Select Rois",frame)
        print(bbox)
        bbox = [tuple(x) for x in bbox]
        bbox_dict[s] = bbox

    cv2.destroyAllWindows()
    return bbox_dict

In [8]:
def track(folder, tracker, bbox, output_file):
    '''
    Track an object initially denoted by a bounding box (bbox)
    and write out the location in an output file
    '''
    seq = cv2.VideoCapture(f"{folder}/Image#%04d.jpg")
    if not seq.isOpened():
        print('error reading image sequence')
    valid, frame = seq.read()
    if not valid:
        print('error reading image')

    # Initialize tracker with first frame and bounding box
    valid = tracker.init(frame, bbox)

    data = []
    while True:
        # Read a new frame
        valid, frame = seq.read()
        if not valid:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        valid, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if valid:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        x = [timer]
        x.extend(bbox)
        # data
        data.append(np.array(x))
        # Display result
        cv2.imsave()
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):  # if press SPACE bar
            break
    
    # Save data
    np.savetxt(output_file, data, delimiter=',')
    cv2.destroyAllWindows()

In [9]:
def dict_to_json(dict, filename):
    '''Saves a dictionary to a json file'''
    with open(filename, 'w') as f:
        json.dump(dict, f)

def json_to_dict(filename):
    '''Loads a json file into a dictionary'''
    with open(filename) as f:
        return json.load(f)


In [10]:
def dict_to_txt(dict, filename):
    '''Saves a dictionary to a txt file'''
    with open(filename, 'w') as f:
        for key in dict:
            f.write(f"{key}:{dict[key]}\n")

def txt_to_dict(filename):
    '''Converts a txt file into a dictionary'''
    with open(filename) as f:
        return {line.split(':')[0]: ast.literal_eval(line.split(':')[1][:-1]) for line in f}

# Track the Blobs

In [11]:
# Pre-processing
set_list = get_folder_list('data')
bbox_dict = get_BB('data', set_list)
dict_to_txt(bbox_dict, 'bbox_dict.txt')

()
()
()


In [12]:
# Tracking
bbox_dict = txt_to_dict('bbox_dict.txt')
tracker = cv2.TrackerCSRT_create()

for s in set_list:
    i = 0
    for bbox in bbox_dict[s]:
        print(s, i, bbox)
        tracker = cv2.TrackerCSRT_create()
        output = track(f"data/{s}", tracker, bbox, f"output/{s}-{i}.txt")
        i += 1

KeyError: 'Set2a'